### lodlitparser

In [6]:
import json
import requests
import time
import re
import sys
from io import BytesIO
from zipfile import ZipFile
from SPARQLWrapper import SPARQLWrapper, JSON
# install NLTK, download wordnet31 ('https://github.com/nltk/nltk_data/blob/gh-pages/packages/corpora/wordnet31.zip');
# put the content of 'wordnet31' to 'wordnet' in 'nltk_data/corpora' (there are issues with importing wordnet31 from nltk.corpus)
from nltk.corpus import wordnet as wn
# download OpenDutchWordnet from 'https://github.com/cultural-ai/OpenDutchWordnet'

### Wikidata

In [16]:
def wd(qids:list,lang:str,user_agent:str) -> dict:
    
    """
    Requesting labels, aliases, descriptions of Wikidata entities
    qids: list of entity IDs (str) (requests 50 entities at a time slicing the list)
    lang: str with language code; for example, 'en' or 'nl';
    (see language codes in Wikidata: https://www.wikidata.org/w/api.php?action=help&modules=wbgetentities)
    user_agent: str with user-agent's info; required by Wikidata (see: https://meta.wikimedia.org/wiki/User-Agent_policy)
    Returns a dict: {'QID': {'type': '',
                     'id': 'QID',
                     'labels': {'lang': {'language': 'lang', 'value': ''}},
                     'descriptions': {'lang': {'language': 'lang','value': ''}},
                     'aliases': {'lang': [{'language': 'lang', 'value': ''}
    """
    
    # 'wbgetentities' constant params
    url = "https://www.wikidata.org/w/api.php"
    params = {"action":"wbgetentities",
              "ids":"", # string of entities (max=50)
              "props":"labels|aliases|descriptions",
              "languages":lang,
              "format":"json"}
    headers = {"user-agent":user_agent}

    # - N LOOPS - #

    # if there's a remainder
    if len(qids) % 50 > 0:
        loops = len(qids) // 50 + 1 # add another loop for requests
    else:
        loops = len(qids) // 50

    # - REQUEST LOOPS - #   

    # counters to slice qids

    start_quid_str = 0
    end_quid_str = 0

    for i in range(0,loops):
        ids_string = "" # putting Qs in one string
        end_quid_str = end_quid_str + 50 # max 50 entities per request

        for q in qids[start_quid_str:end_quid_str]:
            ids_string = ids_string + f"{q}|"

        start_quid_str = start_quid_str + 50

        # updating params

        params["ids"] = ids_string.rstrip("|")

        # sending a request
        d = requests.get(url,params=params,headers=headers)
        literals = d.json() # claims per request

    return literals

### AAT

In [45]:
def aat(aat_uri:list,lang:str) -> dict:
    '''
    Querying prefLabel, altLabel, scopeNote, rdfs comments of concepts in AAT;
    Sends SPARQL queries to the AAT endpoint via SPARQLwrapper 
    aat_uri: list of AAT concepts IDs (str) ['ID']
    lang: str 'en' or 'nl'
    Returns a dict with query results: {'ID':{'lang':'en',
                                              'prefLabel':'',
                                              'altLabels':[],
                                              'scopeNote':'',
                                              'prefLabel_comment':'',
                                              'altLabel_comment':''}
    '''
    
    sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")
    
    if lang == 'en':
        lang_code = '300388277'
    if lang == 'nl':
        lang_code = '300388256'
        
    result_dict = {}
        
    for uri in aat_uri:
        
        result_dict[uri] = {}
        
        query_string = '''SELECT ?prefLabel (GROUP_CONCAT(?altLabel;SEPARATOR="#") AS ?altLabels)
        ?scopeNote ?prefLabel_comment ?altLabel_comment
        WHERE {aat:''' + uri + ''' xl:prefLabel ?pL .?pL dcterms:language aat:''' + lang_code + ''';
        xl:literalForm ?prefLabel .
        OPTIONAL {?pL rdfs:comment ?prefLabel_comment . }
        OPTIONAL {aat:''' + uri + ''' xl:altLabel ?aL .
        ?aL dcterms:language aat:''' + lang_code + ''';
        xl:literalForm ?altLabel . 
        OPTIONAL { ?aL rdfs:comment ?altLabel_comment . }}
        OPTIONAL {aat:''' + uri + ''' skos:scopeNote / dcterms:language aat:'''+ lang_code + ''';
        skos:scopeNote / rdf:value ?scopeNote . }}
        GROUP BY ?prefLabel ?scopeNote ?prefLabel_comment ?altLabel_comment'''
        
        sparql.setQuery(query_string)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        
        altLabels = []
        scopeNote = None
        prefLabel_comment = None
        altLabel_comment = None
        
        for result in results['results']['bindings']:
            
            if 'altLabels' in result:
                altLabels = result['altLabels']['value'].split('#')
            if 'scopeNote' in result:
                scopeNote = result['scopeNote']['value']
            if 'prefLabel_comment' in result:
                prefLabel_comment = result['prefLabel_comment']['value']
            if 'altLabel_comment' in result:
                altLabel_comment = result['altLabel_comment']['value']

            result_dict[uri]['lang'] = lang
            result_dict[uri]['prefLabel'] = result['prefLabel']['value']
            result_dict[uri]['altLabels'] = altLabels
            result_dict[uri]['prefLabel_comment'] = prefLabel_comment
            result_dict[uri]['altLabel_comment'] = altLabel_comment
            result_dict[uri]['scopeNote'] = scopeNote
    print(query_string)    
    return result_dict

### NMVW

In [22]:
def nmvw(term_ids:list) -> dict:
    '''
    Getting info about terms by their handle IDs in NMVW-thesaurus
    term_ids: list of term IDs (str)
    Returns a dict with query results: {'ID': {'prefLabel': '',
                                               'altLabel': [],
                                               'notes': [],
                                               'exactMatch': '',
                                               'scheme': ''}}
    '''
    
    # nmvw: importing thesaurus
    path_to_nmvw = 'https://github.com/cultural-ai/wordsmatter/raw/main/NMVW/nmvw_thesaurus.json.zip'
    nmvw_raw = requests.get(path_to_nmvw).content
    nmvw_zip = ZipFile(BytesIO(nmvw_raw))
    nmvw_json = json.loads(nmvw_zip.read(nmvw_zip.infolist()[0]).decode())

    results_nmvw = {}
    
    for term_id in term_ids:
        handle = 'https://hdl.handle.net/20.500.11840/termmaster' + term_id
        results_nmvw[handle] = nmvw_json.get(handle)
        
    return results_nmvw

### Princeton WordNet

In [13]:
def pwn(synsets:list) -> dict:
    '''
    Getting lemmata, definition, examples of a synset
    synsets: a list of synsets IDs (str)
    Return a dict: {'synset_id': {'lemmata': '',
                                   'definition': '',
                                   'examples': []}
    Requires NLTK, wordnet corpus version 3.1
    '''
    
    results_pwn = {}
    
    for s in synsets:
        synset = wn.synset(s)
        lemmata = [l.name() for l in synset.lemmas()]
        definition = synset.definition()
        examples = synset.examples()
        
        # writing results 
        results_pwn[s] = {'lemmata': lemmata,
                         'definition': definition,
                         'examples': examples}
        
    return results_pwn

### ODWN

In [2]:
def odwn(le_ids:list, path_odwn:str) -> dict:
    '''
    Getting Lemma, sense definitions, examples, synset ID, synset definitions of ODWN Lexical Entries
    le_ids: list of Lexical Entries IDs (str)
    path_odwn: str path to the directory with OpenDutchWordnet (not including the module itself, for example 'user/Downloads')
    Returns a dict: {'le_id': {'lemma': '',
                                'sense_def': '',
                                'examples': [],
                                'synset_ID': '',
                                'synset_def': []}
    '''
    # importing ODWN
    sys.path.insert(0,path_odwn)
    from OpenDutchWordnet import Wn_grid_parser
    # creating an instance
    instance = Wn_grid_parser(Wn_grid_parser.odwn)
    
    # importing all synset definitions
    path_to_glosses = "https://raw.githubusercontent.com/cultural-ai/wordsmatter/main/ODWN/odwn_synset_glosses.json"
    synset_glosses = requests.get(path_to_glosses).json()
    
    results_odwn = {}

    for le_id in le_ids:
        le = instance.les_find_le(le_id)
        synset_id = le.get_synset_id()

        sense_def = le.get_definition()
        if sense_def == '':
            sense_def = None

        results_odwn[le_id] = {'lemma': le.get_lemma(),
                                'sense_def': sense_def,
                                'examples': le.get_sense_example(),
                                'synset_ID': synset_id,
                                'synset_def': synset_glosses.get(synset_id)}
    return results_odwn

#### check where a query term is found per resource:
* Wikidata: labels, aliases
* AAT: prefLabel, altLabel
* NMVW: prefLabel, altLabel
* Princeton WordNet: no checking, only lemmata
* ODWN: no checking, only lemma

#### Getting properties where cont terms are used

In [37]:
def get_literals(resource:str) -> str:
    '''
    Getting literal values of related matches
    resource: str name of the resource 'aat', 'wikidata', 'pwn', 'odwn', 'nmvw' 
    path_odwn: str path to the directory with OpenDutchWordnet (not including the module itself, for example 'user/Downloads')
    Saves json files with literal values:
    aat: 'aat_rm_en.json','aat_rm_nl.json'
    wikidata: 'wikidata_rm_en.json','wikidata_rm_nl.json'
    pwn: 'pwn_rm.json'
    odwn: 'odwn_rm.json'
    nmvw: 'nmvw_rm.json'
    
    '''
    
    # importing related matches
    path_to_rm = 'https://github.com/cultural-ai/wordsmatter/raw/main/rm.json'
    related_matches = requests.get(path_to_rm).json()

    if resource == 'aat':
        # AAT in EN
        aat_en = [v['related_matches']['aat'][0] for v in related_matches.values() if v['lang'] == 'en' \
                  and 'None' not in v['related_matches']['aat']]
        aat_en_results = aat(aat_en,'en')
        with open('aat_rm_en.json', 'w') as jf:
            json.dump(aat_en_results, jf)
        print("AAT EN is saved")

        # AAT in NL
        aat_nl = [v['related_matches']['aat'][0] for v in related_matches.values() if v['lang'] == 'nl' \
                  and 'None' not in v['related_matches']['aat']]
        aat_nl_results = aat(aat_nl,'nl')
        with open('aat_rm_nl.json', 'w') as jf:
            json.dump(aat_nl_results, jf)
        print("AAT NL is saved")
    
    if resource == 'wikidata':
        user_agent = input("Enter your user agent details for Wikidata:")
        
        # Wikidata in EN
        wikidata_en = [v['related_matches']['wikidata'][0] for v in related_matches.values() if v['lang'] == 'en' \
                  and 'None' not in v['related_matches']['wikidata']]
        wikidata_en_results = wd(wikidata_en,'en',user_agent)
        with open('wikidata_rm_en.json', 'w') as jf:
            json.dump(wikidata_en_results, jf)
        print("Wikidata EN is saved")

        # Wikidata in NL
        wikidata_nl = [v['related_matches']['wikidata'][0] for v in related_matches.values() if v['lang'] == 'nl' \
                  and 'None' not in v['related_matches']['wikidata']]
        wikidata_nl_results = wd(wikidata_nl,'nl',user_agent)
        with open('wikidata_rm_nl.json', 'w') as jf:
            json.dump(wikidata_nl_results, jf)
        print("Wikidata NL is saved")
        
    if resource == 'pwn':
        # PWN
        pwn_synsets = []

        for v in related_matches.values():
            if v['lang'] == 'en' and 'None' not in v['related_matches']['pwn']:
                pwn_synsets.extend(v['related_matches']['pwn'])

        pwn_results = pwn(pwn_synsets)
        with open('pwn_rm.json', 'w') as jf:
            json.dump(pwn_results, jf)
        print("PWN is saved")
        
    if resource == 'odwn':
        path_odwn = input("Path to your local OpenDutchWordNet directory:")
        #6 ODWN
        odwn_synsets = []

        for v in related_matches.values():
            if v['lang'] == 'nl' and 'None' not in v['related_matches']['odwn']:
                odwn_synsets.extend(v['related_matches']['odwn'])

        odwn_results = odwn(odwn_synsets,path_odwn)
        with open('odwn_rm.json', 'w') as jf:
            json.dump(odwn_results, jf)
        print("ODWN is saved")
        
    if resource == 'nmvw':
        #7 NMVW
        nmvw_handles = [v['related_matches']['nmvw'][0] for v in related_matches.values() if v['lang'] == 'nl' \
                  and 'None' not in v['related_matches']['nmvw']]
        nmvw_results = nmvw(nmvw_handles)
        with open('nmvw_rm.json', 'w') as jf:
            json.dump(nmvw_results, jf)
        print("NMVW is saved")
    
    return ("All files are saved")

In [8]:
#### Identifying properties where cont terms appear

In [ ]:
def get_hits() -> dict:
    '''
    Adding the info about where contentious terms were found to the related matches
    (applicable to the query results from AAT, Wikidata, and NMVW)
    source: str with the path to the json file with related matches 'related_matches.json'
    '''

In [10]:
# importing related matches
with open('/Users/anesterov/reps/wordsmatter/rm.json','r') as jf:
    related_matches = json.load(jf)

In [ ]:
# Testing functions

In [1]:
from LODlitParser import wd

In [2]:
user_agent = "bot testing search"

In [10]:
filter_by_keywords = ['scientific','article']

In [11]:
filter_by_statements = [('P31','Q5'),('P279','Q7187')]

In [9]:
wd.get_search_hits("aboriginal","en",user_agent,[],statements)

{'aboriginal': 7354}

In [4]:
test_search_results = wd.get_lit_by_term("headhunter","en",user_agent)

In [12]:
params = ''

In [13]:
# filtering by keywords
if filter_by_keywords != []:
    keyword_str = ''
    for kw in filter_by_keywords:
        keyword_str = keyword_str + f" -{kw}"
    params = params + keyword_str

# filtering by statements    
if filter_by_statements != []:
    statements_str = ''
    for t in filter_by_statements:
        statements_str = statements_str + f" -haswbstatement:{t[0]}={t[1]}"
    params = params + statements_str

In [14]:
params

' -scientific -article -haswbstatement:P31=Q5 -haswbstatement:P279=Q7187'

In [16]:
wd.get_lit_by_q(['Q363389'],"nl",user_agent)

[{'Q363389': {'type': 'item',
   'id': 'Q363389',
   'labels': {'nl': {'language': 'nl', 'value': 'Adolf Meyer-Abich'}},
   'descriptions': {'nl': {'language': 'nl',
     'value': 'Duits bibliothecaris (1893-1971)'}},
   'aliases': {}}}]

In [3]:
qids = ['Q363389','Q18530542','Q41710']

In [4]:
langs = ['en','nl']

In [4]:
props = ['P31','P279','P1552']

In [5]:
path = "/Users/anesterov/wd/testing/"

In [5]:
wd.get_labels_by_q(qids,langs,user_agent)

en|nl
{'entities': {'Q363389': {'type': 'item', 'id': 'Q363389', 'labels': {'en': {'language': 'en', 'value': 'Adolf Meyer-Abich'}, 'nl': {'language': 'nl', 'value': 'Adolf Meyer-Abich'}}}, 'Q18530542': {'type': 'item', 'id': 'Q18530542', 'labels': {'en': {'language': 'en', 'value': 'Orde Jonathan Wingate'}}}}, 'success': 1}
{'en': 'Adolf Meyer-Abich', 'nl': 'Adolf Meyer-Abich'}
{'en': 'Orde Jonathan Wingate'}


{'labels': {'Q363389': {'en': 'Adolf Meyer-Abich', 'nl': 'Adolf Meyer-Abich'},
  'Q18530542': {'en': 'Orde Jonathan Wingate'}},
 'failed': []}

In [6]:
wd.get_claims(qids,props,user_agent,path)

'Requests are saved in /Users/anesterov/wd/testing/'

In [5]:
wd.find_where_query_term_appears(test_search_results,'en')

{'headhunter': [{'query_term': 'headhunter',
   'lang': 'en',
   'QID': 'Q105877016',
   'prefLabel': 'Who Says Yes When the Headhunter Calls? Understanding Executive Job Search Behavior',
   'aliases': None,
   'description': ['scientific article published in August 2013'],
   'index': 38,
   'found_in': 'prefLabel'},
  {'query_term': 'headhunter',
   'lang': 'en',
   'QID': 'Q114485638',
   'prefLabel': 'Headhunter',
   'aliases': None,
   'description': ['episode of Quincy, M.E. (S6 E10)'],
   'index': 14,
   'found_in': 'prefLabel'},
  {'query_term': 'headhunter',
   'lang': 'en',
   'QID': 'Q114498379',
   'prefLabel': 'Headhunter',
   'aliases': None,
   'description': ['episode of Police Story (S2 E13)'],
   'index': 10,
   'found_in': 'prefLabel'},
  {'query_term': 'headhunter',
   'lang': 'en',
   'QID': 'Q114521710',
   'prefLabel': 'Headhunter',
   'aliases': None,
   'description': ['episode of Kingdom (S3 E4)'],
   'index': 11,
   'found_in': 'prefLabel'},
  {'query_term':

In [3]:
data_path = "/Users/anesterov/wd/29sep/results_w_claims_en.json"
save_path = "/Users/anesterov/wd/jan/"

In [4]:
wd.filter_proper_names(data_path, "en", save_path)

Q109939822 V. de Schouten de Batavia
Q112463576 Mitch Batavia
Q31941779 Andrew Batavia
Q47163413 Monique Batavia
Q90123715 Chelsea Batavia
Q90628351 Aashil A Batavia
Q90628351 Aashil A Batavia
Q90734418 Jason P Van Batavia
Q102110885 Neal Madras
Q105972402 Madras A.Kannan
Q27804537 Giridhar Madras
Q38006024 Monika Madras
Q43159749 Giridhar Madras
Q61100109 Neal Madras
Q61794164 Durgaprasad Madras Rajaraman Iyer
Q7324116 Richard Benyon of Madras
Q75649665 Hannah Madras
Q75649667 John Madras
Q75649667 John Madras
Q75649672 Mary Madras Beamish
Q75649676 Hannah Madras Beamish
Q75875909 Anne Eliza Madras
Q75875909 Anne Eliza Madras
Q75995831 Robert Conner Madras
Q75999068 Charles Madras Hughes-Hallett
Q110766396 Madras A. Kannan
Q110766396 Madras A. Kannan
Q4895501 Bertha Madras
Q5481252 Francis Hastings of Madras
Q6892510 Mohammad Usman of Madras
Q75875941 John Henry Madras
Q75875941 John Henry Madras
Q84405376 Tomasz Madras
Q91555915 Bertha K Madras
Q91555915 Bertha K Madras
Q87891182 Wil

Q549293 Simon White
Q55193702 Mark A. White
Q55193702 Mark A. White
Q56065957 Justin S White
Q56065957 Justin S White
Q56423811 Scott A White
Q56423811 Scott A White
Q56876923 Herman Brenner White
Q56876923 Herman Brenner White
Q56963021 Duanne A. White
Q56963021 Duanne A. White
Q56963021 Duanne A. White
Q56963021 Duanne A. White
Q56963021 Duanne A. White
Q56963021 Duanne A. White
Q56963021 Duanne A. White
Q56963021 Duanne A. White
Q56963021 Duanne A. White
Q57014863 Jonathan White
Q57042423 Iain White
Q57074702 Claire E White
Q57074702 Claire E White
Q57085537 Roger White
Q572735 Stanford White
Q572735 Stanford White
Q57564649 Ben P White
Q57564649 Ben P White
Q57564649 Ben P White
Q57616560 Isabel White
Q57638230 Stephanie A White
Q57638230 Stephanie A White
Q57638230 Stephanie A White
Q57699411 Sarah White
Q57891777 Scott M. White
Q57891777 Scott M. White
Q57896968 Ross G White
Q57896968 Ross G White
Q57969821 Tonya White
Q580411 Matt White
Q58323558 Simon R White
Q58323558 Simon R 

Q42417591 Liam White
Q42876235 Alex W. White
Q42876235 Alex W. White
Q43914784 Andrew White of Markill
Q4392601 Theodore H. White
Q4392601 Theodore H. White
Q443094 Samuel White
Q444202 Horace White
Q47322611 Laura K White
Q47322611 Laura K White
Q4793671 Armond White
Q48996870 Woodrow White
Q5044687 Carole Ita White
Q5044687 Carole Ita White
Q5045054 Caroline Earle White
Q508721 Andrew White
Q508721 Andrew White
Q51703448 Gregory P. White
Q51703448 Gregory P. White
Q52310723 Jarvis White
Q5241020 David White
Q5241020 David White
Q5241020 David White
Q5301910 Douglas R. White
Q5301910 Douglas R. White
Q5301910 Douglas R. White
Q530377 Meg White
Q530377 Meg White
Q5379339 Enoch White Clark
Q5388910 Erik White
Q5409780 Eunice White Beecher
Q5409780 Eunice White Beecher
Q5409780 Eunice White Beecher
Q54612794 John Clifford White
Q5568098 Glen White
Q57613373 Evan C White
Q57613373 Evan C White
Q58637037 Lucie White
Q59535545 Lauren C White
Q59535545 Lauren C White
Q60034553 Rebecca White 

Q388613 LenDale White
Q42292346 Franklin White
Q433961 Robin White
Q4468428 Marjorie White
Q4679971 Adam White
Q47129246 Eryn Mant White
Q4755319 Andrea White
Q4817499 Norval White
Q4817499 Norval White
Q4817499 Norval White
Q4859117 Barbara M. White
Q4859117 Barbara M. White
Q4863991 Barry B. White
Q5162307 Conor White
Q52161474 Henry White
Q5273546 Dick White
Q5273546 Dick White
Q56451581 Leo White
Q56451581 Leo White
Q59217442 Bradley A White
Q59217442 Bradley A White
Q59255788 Justyn Campbell-White
Q59255788 Justyn Campbell-White
Q59673344 Alexander White
Q5992806 Ignatius White
Q60583587 White Kennett
Q6145427 James White
Q63245335 Julie A. White
Q63245335 Julie A. White
Q64592431 Jerry White
Q64938511 Nicholas A White
Q64938511 Nicholas A White
Q6792581 Maunsel White
Q6969753 Nathaniel Martello-White
Q7087933 Oliver White
Q7087933 Oliver White
Q7154340 Paul White
Q7154340 Paul White
Q7351011 Robert White
Q75249219 Margaret Lucia White
Q75249219 Margaret Lucia White
Q75382237 Hami

Q98945777 Thomas F. White
Q98945777 Thomas F. White
Q99238542 DeTrina White
Q99401893 Colin White-Dzuro
Q7966409 Walter Whiter
Q93146380 Daniel Whiter
Q94276644 Edward T. Whiter
Q94276644 Edward T. Whiter
Q94276644 Edward T. Whiter
Q102277059 Beverly Joyce Whitest
Q102672819 LeeAnn Whites
Q108824247 Hildegardt Whites
Q147675 Zara Whites
Q59762224 David Whites
Q51383944 Pharasmanes of Caucasian Albania
Q19542178 Baboo Lal Verma
Q3445200 Baboo Nimal
Q4837984 Baboo Da Silva
Q5278063 Dinesh Baboo
Q61652309 Jangyeswar Baboo
Q65228119 Baboo Lal
Q7655213 Sweet Baboo
Q84984459 Sabyasachi Baboo
Q86973322 Jagdish Baboo Sharma
Q92220995 Sihaam Jardien-Baboo
Q105324129 Brian S. Page
Q105324129 Brian S. Page
Q165467 Jimmy Page
Q165467 Jimmy Page
Q165467 Jimmy Page
Q29643391 Clive Page
Q29643391 Clive Page
Q29643391 Clive Page
Q3035873 Don Page
Q3035873 Don Page
Q3035873 Don Page
Q58623019 Andrew C. Page
Q58623019 Andrew C. Page
Q7356570 Roderic D. M. Page
Q7356570 Roderic D. M. Page
Q7356570 Roderi

Q75285711 Alice Ada Gay
Q75307485 Gay Hardinge
Q75307485 Gay Hardinge
Q75310095 Sylvia Gay Edwardes
Q75310095 Sylvia Gay Edwardes
Q75331457 Betty Gay Ingham
Q75340347 Gay Mary Fison
Q75342327 Susan Gay Hill
Q75345120 Laura Gay Gore-Langton
Q75347761 Caroline Gay Pinckney
Q75407843 Elizabeth Gay
Q75418122 Mabel Ann Gay Tempest Gill
Q75422861 Laurel Gay Sawbridge
Q75445056 Robin Gay McGregor
Q75445718 Lydia Gay
Q75474470 Alexandra Gay Anderson
Q75479073 Linda Gay Blackburne
Q75481729 Anita Gay Christian
Q75486911 Anthea Gay Palairet
Q75573655 Florence Gay Octavia Williams
Q75583897 Cheryl Gay Harvey
Q75589066 Gay Elizabeth Tudor
Q75589066 Gay Elizabeth Tudor
Q75589066 Gay Elizabeth Tudor
Q75595259 Nancy Gay Robertson
Q75611445 Elizabeth Louisa Gay Coghill
Q75635420 Sarah Gay Lisette Cotter
Q75681547 Vivienne Gay Bruce
Q75745579 Lena Gay Hoskyns
Q75754047 Penelope Gay Hunt
Q75771806 Gay Catherine Solomon
Q75836163 Maureen Gay Brereton
Q75859232 John Robert le Gay Solly
Q75861316 Gay Lincl

Q113622174 Curtis W. Black
Q114051315 Lucinda J Black
Q114051315 Lucinda J Black
Q114051315 Lucinda J Black
Q114084648 Judith Lee Black
Q114084648 Judith Lee Black
Q114084648 Judith Lee Black
Q11627 Black Francis
Q11627 Black Francis
Q1238931 Don Black
Q1238931 Don Black
Q15711987 Dorrit Black
Q15711987 Dorrit Black
Q15711987 Dorrit Black
Q180252 Lewis Black
Q180252 Lewis Black
Q184854 Edward, the Black Prince
Q184854 Edward, the Black Prince
Q184854 Edward, the Black Prince
Q20638823 Sandra Black
Q20638823 Sandra Black
Q229128 Cara Black
Q229128 Cara Black
Q232333 Karen Black
Q232333 Karen Black
Q27107465 Sandra Black
Q27107465 Sandra Black
Q286525 Cilla Black
Q2905414 Black M
Q2905414 Black M
Q2905414 Black M
Q2905414 Black M
Q2905414 Black M
Q30347871 Robert E. Black
Q30347871 Robert E. Black
Q30347871 Robert E. Black
Q30347871 Robert E. Black
Q30349386 Karen H. Black
Q30349386 Karen H. Black
Q30349386 Karen H. Black
Q30349386 Karen H. Black
Q30349386 Karen H. Black
Q30349386 Karen 

Q104946987 Eugene H. Black III
Q108752498 Kenneth Oscar Black
Q110651493 Mary Ann Jouett Black
Q111822391 Tracy Black Enders
Q111822391 Tracy Black Enders
Q1405272 Black Ace
Q16066748 Jack Black
Q16215450 Natural Black
Q18709332 Samuel Gay Black
Q18709332 Samuel Gay Black
Q20639838 Thomas Black
Q21070398 Bobby Black
Q21070398 Bobby Black
Q21456130 Jacqueline Black
Q30014909 John Janvier Black
Q30014909 John Janvier Black
Q30014909 John Janvier Black
Q4718376 Alexander Black Robertson
Q4718376 Alexander Black Robertson
Q4790236 Ariann Black
Q48990176 Robert Black
Q50737229 Stefanie Black
Q5183835 Creed Black
Q5183835 Creed Black
Q5315988 Norman Black
Q5315988 Norman Black
Q5718237 Henry Black
Q58375278 Christine Black
Q58375278 Christine Black
Q6134790 James Gow Black
Q6222059 John Black Packer
Q62273308 Kevin Black
Q62273308 Kevin Black
Q63743566 Mary Black
Q6427156 Kole Black
Q75440876 Lilian Annie Black
Q75579562 Lesley Black
Q75861649 Carolyn Elizabeth Black
Q75902897 John Black
Q75

'File is saved. 8552 entities are filtered out.'